In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


import os
if not os.path.isdir('saved_models_{}'.format('acgan')):
    os.mkdir('saved_models_{}'.format('acgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('acgan'), mode='w')


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

import torch.utils.data as Data
class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 4 * 4, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 512)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                # Image labels. 0-9
                img_labels = y_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # The labels of the digits that the generator tries to create an
                # image representation of
                sampled_labels = np.random.randint(0, 10, (batch_size, 1))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, sampled_labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, [valid, img_labels])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss[0]))
                # If at save interval => save generated image samples
                sampleSize = 5000
                if global_step % sample_interval == 0:
                    self.mode_drop( X_test,y_test,sampleSize,global_step)


    def mode_drop(self, x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for num in range(r) for _ in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        1792      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 128)         512       
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 3.216930, acc.: 39.84%, op_acc: 7.81%] [G loss: 2.815848]
epoch:0 step:2 [D loss: 3.285421, acc.: 52.34%, op_acc: 9.38%] [G loss: 2.942232]
epoch:0 step:3 [D loss: 3.244736, acc.: 54.69%, op_acc: 4.69%] [G loss: 3.100149]
epoch:0 step:4 [D loss: 3.165864, acc.: 54.69%, op_acc: 13.28%] [G loss: 3.450594]
epoch:0 step:5 [D loss: 3.158983, acc.: 55.47%, op_acc: 15.62%] [G loss: 3.484594]
epoch:0 step:6 [D loss: 3.191518, acc.: 50.78%, op_acc: 10.16%] [G loss: 3.270636]
epoch:0 step:7 [D loss: 3.101322, acc.: 57.03%, op_acc: 17.19%] [G loss: 3.685805]
epoch:0 step:8 [D loss: 3.300984, acc.: 49.22%, op_acc: 13.28%] [G loss: 3.624923]
epoch:0 step:9 [D loss: 3.100485, acc.: 57.81%, op_acc: 19.53%] [G loss: 3.580890]
epoch:0 step:10 [D loss: 3.198788, acc.: 47.66%, op_acc: 16.41%] [G loss: 3.658558]
epoch:0 step:11 [D loss: 2.914747, acc.: 62.50%, op_acc: 18.75%] [G loss: 4.329795]
epoch:0 step:12 [D loss: 3.050171, acc.: 54.69%, op_acc: 18.75%] [G loss: 4.114166]
epoc

epoch:0 step:101 [D loss: 2.561774, acc.: 92.19%, op_acc: 16.41%] [G loss: 7.787962]
epoch:0 step:102 [D loss: 2.614391, acc.: 89.06%, op_acc: 25.00%] [G loss: 8.066060]
epoch:0 step:103 [D loss: 2.586424, acc.: 89.06%, op_acc: 26.56%] [G loss: 8.331219]
epoch:0 step:104 [D loss: 2.643169, acc.: 89.06%, op_acc: 17.19%] [G loss: 7.625169]
epoch:0 step:105 [D loss: 3.151361, acc.: 65.62%, op_acc: 15.62%] [G loss: 8.803686]
epoch:0 step:106 [D loss: 2.533786, acc.: 91.41%, op_acc: 17.97%] [G loss: 9.146097]
epoch:0 step:107 [D loss: 2.689166, acc.: 83.59%, op_acc: 26.56%] [G loss: 6.351109]
epoch:0 step:108 [D loss: 3.377406, acc.: 52.34%, op_acc: 11.72%] [G loss: 8.993023]
epoch:0 step:109 [D loss: 2.844429, acc.: 81.25%, op_acc: 19.53%] [G loss: 7.854629]
epoch:0 step:110 [D loss: 2.792169, acc.: 78.91%, op_acc: 24.22%] [G loss: 6.143723]
epoch:0 step:111 [D loss: 2.539076, acc.: 86.72%, op_acc: 22.66%] [G loss: 6.928056]
epoch:0 step:112 [D loss: 2.605430, acc.: 92.19%, op_acc: 26.56%]

...

epoch:49 step:38628 [D loss: 0.902892, acc.: 75.00%, op_acc: 88.28%] [G loss: 1.529479]
epoch:49 step:38629 [D loss: 0.845196, acc.: 73.44%, op_acc: 89.06%] [G loss: 1.584016]
epoch:49 step:38630 [D loss: 0.845548, acc.: 75.78%, op_acc: 89.84%] [G loss: 1.612693]
epoch:49 step:38631 [D loss: 0.763332, acc.: 72.66%, op_acc: 91.41%] [G loss: 1.757085]
epoch:49 step:38632 [D loss: 0.831973, acc.: 75.78%, op_acc: 89.06%] [G loss: 2.054763]
epoch:49 step:38633 [D loss: 0.860018, acc.: 70.31%, op_acc: 90.62%] [G loss: 1.825835]
epoch:49 step:38634 [D loss: 0.742782, acc.: 75.78%, op_acc: 91.41%] [G loss: 1.972797]
epoch:49 step:38635 [D loss: 0.781327, acc.: 76.56%, op_acc: 90.62%] [G loss: 1.809225]
epoch:49 step:38636 [D loss: 0.779802, acc.: 75.00%, op_acc: 91.41%] [G loss: 1.610833]
epoch:49 step:38637 [D loss: 0.803550, acc.: 76.56%, op_acc: 89.06%] [G loss: 1.611264]
epoch:49 step:38638 [D loss: 0.833771, acc.: 78.91%, op_acc: 92.97%] [G loss: 1.860372]
epoch:49 step:38639 [D loss: 0.7

epoch:49 step:38723 [D loss: 0.909029, acc.: 67.19%, op_acc: 90.62%] [G loss: 1.569580]
epoch:49 step:38724 [D loss: 0.957780, acc.: 73.44%, op_acc: 88.28%] [G loss: 1.512133]
epoch:49 step:38725 [D loss: 0.904153, acc.: 70.31%, op_acc: 89.84%] [G loss: 1.768658]
epoch:49 step:38726 [D loss: 0.767939, acc.: 75.00%, op_acc: 89.84%] [G loss: 1.700838]
epoch:49 step:38727 [D loss: 0.740385, acc.: 80.47%, op_acc: 90.62%] [G loss: 1.854017]
epoch:49 step:38728 [D loss: 0.933784, acc.: 67.97%, op_acc: 93.75%] [G loss: 1.749145]
epoch:49 step:38729 [D loss: 0.661847, acc.: 79.69%, op_acc: 93.75%] [G loss: 1.582823]
epoch:49 step:38730 [D loss: 0.700030, acc.: 80.47%, op_acc: 91.41%] [G loss: 1.870233]
epoch:49 step:38731 [D loss: 0.697038, acc.: 75.00%, op_acc: 95.31%] [G loss: 1.581398]
epoch:49 step:38732 [D loss: 0.927275, acc.: 64.06%, op_acc: 91.41%] [G loss: 1.788198]
epoch:49 step:38733 [D loss: 0.868786, acc.: 70.31%, op_acc: 90.62%] [G loss: 1.199433]
epoch:49 step:38734 [D loss: 0.9

epoch:49 step:38818 [D loss: 0.834472, acc.: 72.66%, op_acc: 88.28%] [G loss: 1.665961]
epoch:49 step:38819 [D loss: 0.986098, acc.: 73.44%, op_acc: 85.16%] [G loss: 1.752926]
epoch:49 step:38820 [D loss: 0.695256, acc.: 72.66%, op_acc: 93.75%] [G loss: 1.581492]
epoch:49 step:38821 [D loss: 0.823793, acc.: 75.00%, op_acc: 93.75%] [G loss: 1.559355]
epoch:49 step:38822 [D loss: 0.876553, acc.: 66.41%, op_acc: 89.84%] [G loss: 1.387816]
epoch:49 step:38823 [D loss: 0.825090, acc.: 68.75%, op_acc: 85.94%] [G loss: 1.605075]
epoch:49 step:38824 [D loss: 0.639318, acc.: 75.78%, op_acc: 97.66%] [G loss: 1.828084]
epoch:49 step:38825 [D loss: 0.881367, acc.: 65.62%, op_acc: 91.41%] [G loss: 1.440075]
epoch:49 step:38826 [D loss: 1.018463, acc.: 65.62%, op_acc: 85.94%] [G loss: 1.504706]
epoch:49 step:38827 [D loss: 0.756964, acc.: 80.47%, op_acc: 92.97%] [G loss: 1.785169]
epoch:49 step:38828 [D loss: 0.847391, acc.: 69.53%, op_acc: 88.28%] [G loss: 1.650097]
epoch:49 step:38829 [D loss: 0.8

epoch:49 step:38912 [D loss: 0.894707, acc.: 74.22%, op_acc: 86.72%] [G loss: 1.771716]
epoch:49 step:38913 [D loss: 0.676196, acc.: 72.66%, op_acc: 93.75%] [G loss: 1.694068]
epoch:49 step:38914 [D loss: 0.767372, acc.: 75.78%, op_acc: 90.62%] [G loss: 1.565087]
epoch:49 step:38915 [D loss: 1.019439, acc.: 71.09%, op_acc: 83.59%] [G loss: 1.808765]
epoch:49 step:38916 [D loss: 0.716509, acc.: 78.91%, op_acc: 93.75%] [G loss: 1.754006]
epoch:49 step:38917 [D loss: 0.846353, acc.: 77.34%, op_acc: 85.94%] [G loss: 1.669517]
epoch:49 step:38918 [D loss: 0.780302, acc.: 75.00%, op_acc: 90.62%] [G loss: 1.630026]
epoch:49 step:38919 [D loss: 1.133365, acc.: 67.19%, op_acc: 86.72%] [G loss: 1.647365]
epoch:49 step:38920 [D loss: 0.784367, acc.: 73.44%, op_acc: 90.62%] [G loss: 1.414481]
epoch:49 step:38921 [D loss: 0.793362, acc.: 70.31%, op_acc: 90.62%] [G loss: 1.612669]
epoch:49 step:38922 [D loss: 0.899545, acc.: 67.19%, op_acc: 92.97%] [G loss: 1.671243]
epoch:49 step:38923 [D loss: 0.8

epoch:49 step:39006 [D loss: 0.713586, acc.: 77.34%, op_acc: 92.19%] [G loss: 1.766236]
epoch:49 step:39007 [D loss: 0.874591, acc.: 69.53%, op_acc: 91.41%] [G loss: 1.801821]
epoch:49 step:39008 [D loss: 0.761983, acc.: 73.44%, op_acc: 92.19%] [G loss: 1.656850]
epoch:49 step:39009 [D loss: 0.721235, acc.: 75.78%, op_acc: 92.97%] [G loss: 2.210772]
epoch:49 step:39010 [D loss: 0.725855, acc.: 78.12%, op_acc: 91.41%] [G loss: 1.515471]
epoch:49 step:39011 [D loss: 0.722091, acc.: 69.53%, op_acc: 92.19%] [G loss: 1.744312]
epoch:49 step:39012 [D loss: 0.805742, acc.: 77.34%, op_acc: 87.50%] [G loss: 1.616399]
epoch:49 step:39013 [D loss: 0.780134, acc.: 62.50%, op_acc: 92.19%] [G loss: 1.346272]
epoch:49 step:39014 [D loss: 0.757722, acc.: 76.56%, op_acc: 91.41%] [G loss: 1.630731]
epoch:49 step:39015 [D loss: 0.962448, acc.: 78.12%, op_acc: 86.72%] [G loss: 1.729516]
epoch:49 step:39016 [D loss: 0.796377, acc.: 69.53%, op_acc: 92.97%] [G loss: 1.795244]
epoch:49 step:39017 [D loss: 0.8